<i><b>Public AI</b></i>
<br>
# ResNet 논문 구현하기

### _Objective_

* Residual Network의 논문을 읽고 구현하면서, ResNet의 모델을 이해해 보겠습니다. <br>


In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf

<br><br>

# \[ Paper Implementation \]
---


VGG Network처럼 매우 간단한 형태로, Inception Network에 비해 짜기가 매우 간단합니다.

Reference : [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)

<br>

## 1. Placeholder 구성하기


<img src="https://i.imgur.com/1MyBu1b.png" width="500">

* VGG Network와 마찬가지로 이미지의 크기는 (224,224,3)이며, 전처리로서는 Image의 Mean Value를 빼주는 방식으로 진행되었습니다.

In [42]:
from tensorflow.keras.layers import Input

input_shape = (224,224,3)

inputs = Input(input_shape, name='images')

rgb_mean = np.array([123.68, 116.779, 103.939], np.float32)
preprocessed = (inputs - rgb_mean)

<br>

## 2. Inference Network 구성하기

![Imgur](https://i.imgur.com/ajlup9L.png)
* ResNet에서 핵심은 실선과 점선으로 이루어진 Residual Block입니다.<br>
* 실선으로 이루어진 부분은 Input과 Output의 shape가 같아서, 바로 더해줄 수 있는 Block을 의미하고<br>
점선으로 이루어진 부분은 Input과 Output의 Shape가 달라서, 바로 더해줄 수 없고 stride와 Projection을 통해 Shape을 동일하게 해주어야 합니다.

![Imgur](https://i.imgur.com/KRR62oi.png)

ResNet-34을 구성해보도록 하겠습니다.

### (1) conv2_x까지 구성하기

그림에는 나타나 있지 않지만, ResNet의 모든 Convolution Layer 다음에는 Batch Normalization Layer이 뒤따라 옵니다.

<img src="https://i.imgur.com/g3zZGs7.png" width="500">

In [43]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU

In [44]:
conv = Conv2D(64, (7,7), strides=(2,2),padding='SAME')(preprocessed)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
out = MaxPooling2D((3,3),(2,2),padding='SAME')(act)

# conv2_1
conv = Conv2D(64, (3,3), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(64, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = out + bn
out = ReLU()(added)

# conv2_2
conv = Conv2D(64, (3,3), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(64, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = out + bn
out = ReLU()(added)

# conv2_3
conv = Conv2D(64, (3,3), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(64, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = out + bn
out = ReLU()(added)

### (2) Conv3_x 구성하기
<img src="https://i.imgur.com/ctUSU6L.png" width="500">

conv2 block에서 conv3 block으로 넘어갈 때, Input과 output의 크기가 달라집니다.<br>
1. Feature Map의 갯수가 커지는 것 : 1x1 Conv로 차원을 늘리는 것
2. Feature Map의 size가 줄어드는 것 : Stride를 통해 줄임

In [46]:
projection = tf.layers.Conv2D(128,(1,1),strides=(2,2))(out)

conv = Conv2D(128, (3,3), strides=(2,2), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(128, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = projection + bn
out = ReLU()(added)

conv = Conv2D(128, (3,3), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(128, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = out + bn
out = ReLU()(added)

conv = Conv2D(128, (3,3), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(128, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = out + bn
out = ReLU()(added)

conv = Conv2D(128, (3,3), padding='SAME')(out)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
conv = Conv2D(128, (3,3), padding='SAME')(act)
bn = BatchNormalization()(conv)
added = out + bn
out = ReLU()(added)

크게 두 가지 유형의 Residual이 존재한다는 것을 알 수 있습니다.<br>
Input과 Output의 크기가 동일한 residual block(실선으로 표시)과 <br>
Input과 Output의 크기가 다른 residual block(점선으로 표시)로 나뉘어집니다.

우리는 이렇게 동일한 패턴으로 되어있을 경우, 메소드로 작성하는 것이 보다 간결하고<br>
안전하게 코드를 구성할 수 있습니다.

In [47]:
def residual_block(input_layer, filters, strides=(1,1)):
    if input_layer.shape.as_list()[-1] != filters:
        # input_layer의 필터 갯수와 filters가 다르면, projection layer을 거침
        projection = Conv2D(
            filters, (1,1), strides=strides, padding='SAME')(input_layer)
    else:
        # 동일하면 바로 이어줌
        projection = input_layer
        
    conv = Conv2D(filters, (3,3), strides, padding='SAME')(input_layer)
    bn = BatchNormalization()(conv)
    act = ReLU()(bn)
    conv = Conv2D(filters, (3,3), padding='SAME')(act)
    bn = BatchNormalization()(conv)
    added = projection + bn
    out = tf.nn.relu(added)       

    return out        

### (3) 전체 Residual Block 구성하기

위에 구성한 메소드를 이용해 Residual Block을 conv5_x까지 마저 구하겠습니다.

In [48]:
input_shape = (224,224,3)
num_classes = 1000

inputs = Input(input_shape, name='images')

rgb_mean = np.array([123.68, 116.779, 103.939], np.float32)
preprocessed = (inputs - rgb_mean)

conv = Conv2D(64, (7,7), strides=(2,2),padding='SAME')(preprocessed)
bn = BatchNormalization()(conv)
act = ReLU()(bn)
out = MaxPooling2D((3,3),(2,2),padding='SAME')(act)

res2_1 = residual_block(out, 64)
res2_2 = residual_block(res2_1, 64)    
res2_3 = residual_block(res2_2, 64) 

res3_1 = residual_block(res2_3, 128, strides=(2,2))
res3_2 = residual_block(res3_1, 128)    
res3_3 = residual_block(res3_2, 128)    
res3_4 = residual_block(res3_3, 128)    

res4_1 = residual_block(res3_4, 256, strides=(2,2))
res4_2 = residual_block(res4_1, 256)    
res4_3 = residual_block(res4_2, 256)    
res4_4 = residual_block(res4_3, 256)    
res4_5 = residual_block(res4_4, 256)    
res4_6 = residual_block(res4_5, 256)

res5_1 = residual_block(res4_6, 512, strides=(2,2))
res5_2 = residual_block(res5_1, 512)    
res5_3 = residual_block(res5_2, 512)        

### (4) Global Average Pooling Layer 구성하기

![Imgur](https://i.imgur.com/kbVvJCU.png)

ResNet의 마지막 층에서는 Global Average Pooling Layer가 붙습니다.<br>
Global Average Pooling Layer는 각 필터 층 별로 평균값을 산출하는 것을 의미합니다.<br>

Global Average Pooling Layer가 붙을 경우, Input Image의 크기에 무관하게, 분류기에 넣을 수 있게 됩니다.<Br>
ResNet 이후 많은 모델들은 Global Average Pooling Layer를 Classification 모델 마지막에 넣음으로써 이미지의 크기에 무관한 모델로 구현하였습니다.

In [49]:
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense

In [50]:
gap = GlobalAveragePooling2D()(res5_3)
pred = Dense(num_classes, activation='softmax')(gap)

### (5) 모델 구성하기

In [ ]:
from tensorflow.keras.models import Model

In [52]:
model = Model(inputs, pred)

model.summary()

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/05/14

---